# Preparation

In [ ]:
import subprocess
import threading

#istallazione di ollama
!curl -fsSL https://ollama.com/install.sh | sh# This Python 3 environment comes with many helpful analytics libraries installed

In [ ]:
def start_ollama():
    t = threading.Thread(target=lambda: subprocess.run(["ollama", "serve"]),daemon=True)
    t.start()

In [ ]:
def pull_model(local_llm):
    !ollama pull local_llm

In [ ]:
def start_model(local_llm):        
    t2 = threading.Thread(target=lambda: subprocess.run(["ollama", "run", local_llm]),daemon=True)
    t2.start()

In [ ]:
import os

os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from tqdm import tqdm

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
import torch

device = 0 if torch.cuda.is_available() else -1

# response: biased or non-biased
bias_model_tokenizer = AutoTokenizer.from_pretrained("d4data/bias-detection-model")
bias_model = AutoModelForSequenceClassification.from_pretrained("d4data/bias-detection-model",from_tf=True)

# Bias detection

## Dati

In [ ]:
# todo

x_test = None
y_test = None

## Test

In [ ]:
model = "llama3.1"

start_ollama()
pull_model(model_name)
start_model(model_name)

In [ ]:
prompt = PromptTemplate(
        template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> 
        You will be provided with a text, and your task is to classify it into 
        one of two categories: "biased" or "non-biased." A text is considered "biased" 
        if it expresses a personal opinion, uses emotional or partial language, or promotes 
        a particular perspective at the expense of others. A text is considered "non-biased" 
        if it presents facts in a neutral and objective manner, without evident influences.
        Give ONLY the class "biased" or "non-biased", NO PREAMBLE, NO EXPLANATIONS.
        Below, I will show you some classification examples to help you understand 
        how to distinguish between the two types of texts. \n

        Example 1: \n
        Text: "The new educational program is a complete failure, designed with no regard for the needs of the students." \n
        Answer: biased \n

        Example 2: \n
        Text: "The new educational program was introduced with the goal of improving student performance in scientific subjects." \n
        Answer: non-biased \n

        Example 3: \n
        Text: "The current government has done a terrible job managing the economic crisis, causing more harm than good." \n
        Answer: biased \n

        Example 4: \n
        Text: "The government has implemented a series of measures to address the economic crisis, with mixed results according to various analysts." \n
        Answer: non-biased \n
   
         <|eot_id|><|start_header_id|>user<|end_header_id|> 
        Text: {text} \n
        Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
        input_variables=["text"],
    )

In [ ]:
# response: biased or non-biased

def bias_det(prompt, model):
    llm = ChatOllama(model=model, temperature=0)
    bias_det_chain = prompt | llm | StrOutputParser()
    return bias_det_chain

In [ ]:
# Label 0: biased
# Label 1: non-biased

def predict(prompt,model,x_test,encoder):
    y_pred = []
    if encoder:
        bias_detection = pipeline('text-classification', model=bias_model, tokenizer=bias_model_tokenizer, device=device) # cuda = 0,1 based on gpu availability
    for x in tqdm(x_test):
        if encoder:
            answer = bias_detection(x)
        else:
            answer = hate_speech_detection(prompt,model).invoke({"text":x})
        if answer.lower() == "biased": y_pred.append(0)
        elif answer.lower() == "non-biased": y_pred.append(1)
    return y_pred

# True se vogliamo il modello encoder, False se vogliamo usare LLM
encoder = True
y_pred = predict(prompt,model,x_test,encoder)

In [ ]:
# Valutazione del modello
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Classification Report:\n{class_report}")

# Debiasing

In [ ]:
#todo predisposizione dati e metriche di utilizzo

In [ ]:
def debiasing_answer(local_llm):
    prompt = PromptTemplate(
        template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> The user will pass you text cointaining biases.
        Your role is to give a debiased version of the text. Give ONLY the debiased text, NO PREAMBLE, NO EXPLANATIONS.
        Remember that we should treat people from different socioeconomic statuses, sexual orientations, religions, races, physical appearances, nationalities,
        gender identities, disabilities, and ages equally. When we do not have sufficient information, we should choose the unknown option, rather
        than making assumptions based on our stereotypes.
         <|eot_id|><|start_header_id|>user<|end_header_id|> 
        {text}
        <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
        input_variables=["text"],
    )
    llm = ChatOllama(model=local_llm, temperature=0)
    debiasing = prompt | llm | StrOutputParser()
    return debiasing